<a href="https://colab.research.google.com/github/sahana0007/Spark/blob/main/PySpark_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 56.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=8981ab92fadab43013f78968ebc9d1245a92804105cdc190a1c1504355d86f19
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark import SparkContext, SparkConf

In [ ]:
import pyspark
from pyspark.sql import SparkSession
#spark = SparkSession.builder.master("local").appName("RDD_Practise").getOrCreate()

In [ ]:
conf = SparkConf().setAppName("RDD_Practise1").setMaster("local")
sc = SparkContext (conf = conf)

In [ ]:
rdd = sc.parallelize([5,5,4,3,2,9,2])


RDD ACTIONS


In [ ]:
rdd.sum()

30

In [ ]:
rdd.take(3)

[5, 4, 3]

In [ ]:
rdd.top(4)

[9, 5, 4, 3]

In [ ]:
rdd = rdd.distinct()
print(rdd.collect())


[5, 4, 3, 2, 9]


In [ ]:
rdd1 = sc.parallelize(['A', 'A', 'B'])
rdd2 = rdd1.distinct()
print(rdd1.collect())
print(rdd2.collect())

['A', 'A', 'B']
['A', 'B']


In [ ]:
type(rdd1)

pyspark.rdd.RDD

In [ ]:
dir(rdd1)

In [ ]:
help(rdd1)

ACTIONS

In [ ]:
rdd.countByValue()

defaultdict(int, {2: 2, 3: 1, 4: 1, 5: 2, 9: 1})

In [ ]:
rdd1 = sc.parallelize([('A',2), ('B',6), ('C', 8)])

In [ ]:
rdd1.countByKey()

defaultdict(int, {'A': 1, 'B': 1, 'C': 1})

In [ ]:
rdd1.countByValue()

defaultdict(int, {('A', 2): 1, ('B', 6): 1, ('C', 8): 1})

In [ ]:
rdd = sc.parallelize([5,5,4,3,2,9,2])
rdd.reduce(lambda a,b: a+b)

30

In [ ]:
from operator import add
rdd1_fold = sc.parallelize([1,2,3,4,7,10])
rdd1_fold.fold(0,add)

27

In [ ]:
rdd1_range = sc.parallelize(range(1,10))
rdd1_range.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
rdd.collect()

[5, 5, 4, 3, 2, 9, 2]

In [ ]:
rdd.coalesce(1).saveAsTextFile("file:///F:/Spark/save/saveasfile.csv")


RDD TRANSFORMATIONS

In [ ]:
from google.colab import files
files.upload()

MAP

In [ ]:
x_map = sc.parallelize([1,2,5,7,9,10,33])
y_map = x_map.map(lambda x: x**2)

#collect the copies of the elements to a list
print('values of x_map: {0}'.format(x_map.collect()))
print('values of y_map: {0}'.format(y_map.collect()))

values of x_map: [1, 2, 5, 7, 9, 10, 33]
values of y_map: [1, 4, 25, 49, 81, 100, 1089]


FILTER

In [ ]:
x_rdd = sc.parallelize([1,2,5,7,9,10,33])
y_filter = x_rdd.filter(lambda x: x%2 == 0)

#collect the copies of the elements to a list
print('Values of x_rdd: {0}'.format(x_rdd.collect()))
print('values of y_filter: {0}'.format(y_filter.collect()))


Values of x_rdd: [1, 2, 5, 7, 9, 10, 33]
values of y_filter: [2, 10]


FLATMAP

In [ ]:
x_flatmap = sc.parallelize([1,2,7,9,10,11])
y_flatmap = x_flatmap.flatMap(lambda x: (x**2, x+1,100*x))

#collect the copies of the elements to a list
print('values of x_flatmap: {0}'.format(x_flatmap.collect()))
print('values of y_flatmap: {0}'.format(y_flatmap.collect()))

values of x_flatmap: [1, 2, 7, 9, 10, 11]
values of y_flatmap: [1, 2, 100, 4, 3, 200, 49, 8, 700, 81, 10, 900, 100, 11, 1000, 121, 12, 1100]


In [ ]:
x_map_partition = sc.parallelize([1,2,7,9,10,11],2)
def f(iterator): yield sum(iterator)
y_map_partition = x_map_partition.mapPartitions(f)

#glom flattens elements on same partitions
print(x_map_partition.glom().collect())
print(y_map_partition.glom().collect())

[[1, 2, 7], [9, 10, 11]]
[[10], [30]]


Map Partitions With Index

In [ ]:
x_ind = sc.parallelize([1,2,7,9,10,11],3)
def f(partitionIndex, iterator): yield (partitionIndex,sum(iterator))
y_ind = x_ind.mapPartitionsWithIndex(f)

#glom flattens elements on same partitions
print(x_ind.glom().collect())
print(y_ind.glom().collect())

[[1, 2], [7, 9], [10, 11]]
[[(0, 3)], [(1, 16)], [(2, 21)]]


Union

In [ ]:
rdd_1 = sc.parallelize([1,1,2,3,4])
rdd_2 = sc.parallelize([1,1,2,4,5])

rdd_1.union(rdd_2).collect()

[1, 1, 2, 3, 4, 1, 1, 2, 4, 5]

Intersection

In [ ]:
rdd_1 = sc.parallelize([1,1,2,3,4])
rdd_2 = sc.parallelize([1,1,2,4,5])

rdd_1.intersection(rdd_2).collect()

[2, 4, 1]

Distinct

In [ ]:
rdd_1 = sc.parallelize(['a','a','b','c','d'])
rdd_2 = rdd_1.distinct()

print('Values of x: {0}'.format(rdd_1.collect()))
print('Dictinct values of y: {0}'.format(rdd_2.collect()))

Values of x: ['a', 'a', 'b', 'c', 'd']
Dictinct values of y: ['a', 'b', 'c', 'd']


GroupbyKey

In [ ]:
rdd_v = sc.parallelize([('A',2), ('A',6) ,('B',6),('B',9), ('C', 8)])
rdd_gpk = rdd_v.groupByKey()

print(rdd_v.collect())
print([(j[0],[i for i in j[1]]) for j in rdd_gpk.collect()])

[('A', 2), ('A', 6), ('B', 6), ('B', 9), ('C', 8)]
[('A', [2, 6]), ('B', [6, 9]), ('C', [8])]
